In [42]:
import json
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
import xml.etree.ElementTree as ET
import re

In [19]:
headers = {'User-Agent': "dmbernaal@gmail.com"}

In [24]:
response = requests.get('https://www.sec.gov/files/company_tickers.json', headers=headers)
tickers = pd.DataFrame(json.loads(response.text)).T

In [26]:
def ticker2cik(ticker): return tickers[tickers['ticker'] == ticker]['cik_str'].astype(str).values[0].zfill(10)

In [27]:
appl_cik = ticker2cik('AAPL')
appl_cik

'0000320193'

In [88]:
def download_sec_data(cik, type, start=0, count=50):
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type={type}&start={start}&count={count}&owner=exclude&output=atom"
    response = requests.get(url, headers=headers)
    print(response.status_code)
    if response.status_code != 200: return None
    return response.text

In [89]:
appl_sec_data = download_sec_data(appl_cik, '10-Q')

200


In [90]:
# find filing links
soup = BeautifulSoup(appl_sec_data, 'html.parser')
filing_links = soup.find_all('filing-href')


# grab the link from each filing
filing_links = [link.string for link in filing_links]

In [91]:
filing_links

['https://www.sec.gov/Archives/edgar/data/320193/000032019323000077/0000320193-23-000077-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000064/0000320193-23-000064-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000006/0000320193-23-000006-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000070/0000320193-22-000070-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000059/0000320193-22-000059-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000007/0000320193-22-000007-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000065/0000320193-21-000065-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000056/0000320193-21-000056-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000010/0000320193-21-000010-index.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000062/0000320193-20-000062-in

In [13]:
# # FOR LARGE BULK DOWNLOADS USE THIS:
# dl = Downloader("Odyssey", "dmbernaal@gmail.com")
# dl.get("10-Q", "AAPL", after="2023-01-01", download_details=True)